# Měření velikosti zahřívané součátky
Tento úkol je zaměřen na využití kombinace dat z obyčejné RGB kamery a termokamery. Jako zahřívaná součástka bylo využito zařízení **Raspberry Pi 2 (Model B)** společně s **parazitními 2A rezistory** připojeny v USB.

Úkol má za cíl oživit dovednosti studentů při práci s obrazem, jakými jsou například projektivní transformace, segmentace nejen v barevných kanálech nebo měření velikostí v reálných jednotkách.

<img src="images/vv.png" width="50%"/>

### Import knihoven a konfigurace

In [ ]:
%run ../../../tutorials/files/svz.ipynb

from scipy.spatial import distance

### Pomocné funkce
Seznamte se, lze využít. Ostatní již známé funkce jsou v notebooku [svz](../../tutorials/svz.ipynb).

In [ ]:
def select_image_points(img, points_cnt = 4):
    """ Opens a new image window, where user can interactively add or remove image points.
    Points are added while holding CTRL key + pressing left mouse button and removed by ALT key + pressing left mouse button.
    The point selection is terminated by pressing the 'q' key.
    
    Parameters
    ----------
    img : ndarray
        Input image where image points are choosen and drawn.
    points_cnt : Optional[int]
        A maximum number of points to choose. A minimum number of points to compute the projective transformation is 4.
    Returns
    -------
    list
        Returns a list of size >= 4 and size <= points_cnt such that each elements represent (x, y) coordinate in input image.
    """
    if points_cnt < 4: 
        raise ValueError('Number of points must be >= 4.')
        
    points = []
    window_name = 'Point selection'
    img_dimensions = img.shape[:2]
    pts_dist_thresh = 0.01 * img_dimensions[1] # Scale drawing elements with image size
    
    def draw_circle(event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            if flags == cv2.EVENT_FLAG_ALTKEY + cv2.EVENT_LBUTTONDOWN: 
                for p in points:
                    if distance.euclidean(p, (x, y)) < pts_dist_thresh:
                        points.remove(p)
                        break
            elif flags == cv2.EVENT_FLAG_CTRLKEY + cv2.EVENT_LBUTTONDOWN and len(points) < points_cnt:
                points.append((x, y))           

    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL | cv2.WINDOW_GUI_NORMAL)
    cv2.resizeWindow(window_name, 1080, 720)
    cv2.moveWindow(window_name, 0, 0)
    cv2.setMouseCallback(window_name, draw_circle)
    
    # Drawing section, scale drawing elements with image size
    circle_diam = int(0.003 * img_dimensions[1])
    lbl_offset = int(0.005 * img_dimensions[1])
    lbl_font_scale = (0.001 * img_dimensions[1])
    lbl_thickness = int(0.003 * img_dimensions[1])
    
    while 1:
        drawn_img = img.copy()
        
        for i, p in enumerate(points):
            cv2.circle(drawn_img, p, circle_diam , (0, 0, 255), cv2.FILLED)
            cv2.putText(drawn_img, str(i), (p[0] + circle_diam + lbl_offset, p[1] + circle_diam + lbl_offset),
                        0, lbl_font_scale, (0, 0, 255), lbl_thickness)
            
        cv2.imshow(window_name, drawn_img)
        k = cv2.waitKey(1) & 0xFF
        
        if k == ord('q'):
            break

    cv2.destroyAllWindows()
    
    if len(points) < 4: 
        raise ValueError('Number of choosen points must be >= 4.')
        
    return points

def copy_to(src, dst, mask):
    '''Python alternative to C++/Java OpenCV's Mat.copyTo().
    More: https://docs.opencv.org/trunk/d3/d63/classcv_1_1Mat.html#a626fe5f96d02525e2604d2ad46dd574f'''
    locs = np.where(mask != 0) # Get the non-zero mask locations
    dst[locs[0], locs[1]] = src[locs[0], locs[1]]
    return dst

def show_scale(min_val, max_val, color_map='jet'):
    fig = plt.figure(figsize=(8, 3))
    ax1 = fig.add_axes([0.05, 0.80, 0.9, 0.15])
    norm = matplotlib.colors.Normalize(vmin=min_val, vmax=max_val)
    cb1 = matplotlib.colorbar.ColorbarBase(ax1, cmap=color_map, norm=norm, orientation='horizontal')
    cb1.set_label('Temperature °C')
    plt.show()

In [ ]:
def normalized_image(img):
    # Dejte si pozor, že cv2.normalize nalezne minimální/maximální hodnotu v obraze vůči kterým normalizuje a tento vstup nelze změnit.
    # Můžeme pouze nastavit nové minimum a maximum. Později v úkolu budete muset napsat svoji "chytřejší" normalizaci.
    scaled = np.zeros_like(img)
    cv2.normalize(img, scaled, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    return scaled.astype('uint8')

def to_3_channels(image_one_channel):
    h,w = image_one_channel.shape
    image3 = np.zeros((h,w,3), dtype=np.uint8)
    image3[:,:,0] = image_one_channel
    image3[:,:,1] = image_one_channel
    image3[:,:,2] = image_one_channel
    return image3

def load_termo_csv(file_path):
    return np.genfromtxt(file_path, delimiter=';')[:, :-1]

---

## Úkol
K obrazové práci nad zahřívanou součástky jsou k dispozici nasnímaná data z rgb kamery a data reálných teplot z termokamery. Oboje je dostupné ve složce `data/`.

Za úkol je možné získat až **8 bodů** a až **2 prémiové**.

### 1) Načtěte data
Načtěte obrazová data ve formátu `png` a data teplot ve formátu `csv`. Data teplot obsahují reálné hodnoty teplot v jednotkách **°C** s přesností na desetinu stupně. Vhodně zobrazte jak data **obrazová**, tak data **teplot** obrázkem.

<div style="color: blue; text-align: right">[ 0,5 bodu ]</div>

In [ ]:
DEBUG = True

In [ ]:
image_rgb = cv2.imread('data/data_rgb.png')
image_termo = load_termo_csv('data/data_temps.csv')
image_termo_norm = normalized_image(image_termo)
plot_images(image_termo_norm, image_rgb)

### 2) Vypište hodnoty několika teplot
Zvolte v obraze **3 body** a vypište hodnoty teploty v těchto bodech. Snažte se najít místa s **nejvyššími** teplotami.

<div style="color: blue; text-align: right">[ 0,5 bodu ]</div>

In [ ]:
print(image_termo[300][500])
print(image_termo[328][300])
print(image_termo[328][200])

### 3) Namapujte termosnímek na RGB
Uvědomte si, že snímky **nejsou stejně velké** ani foceny ze **stejného úhlu**. Pamatujte, že je **důležité** uvědomit si **správně rovinu**, ve které pracujeme. Zobrazte jak obrázek RGB, tak namapovaný termosnímek.

Klíčová slova:
- přiřazení dvojic bodů
- hledání transformační matice
- warpování termosnímku na perspektivu RGB

<div style="color: blue; text-align: right">[ 1,5 bodu ]</div>

In [ ]:
#Detects holes for screws in rapsberry pi image using morphology and HuoughCircles
#preimum 2
def screw_holes_rgb(image, lower_bound, upper_bound):
    
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv_image, lower_bound, upper_bound)
    #remove noise
    kernel = np.ones((3,3),np.uint8)
    mask_opened = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    #prepare for better detection
    kernel = np.ones((30,10),np.uint8)
    mask_closed = cv2.morphologyEx(mask_opened, cv2.MORPH_CLOSE, kernel)
    
    image = cv2.bitwise_and(hsv_image, hsv_image, mask=mask_closed)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    if DEBUG:
        plot_images(mask, mask_opened, mask_closed, image)
    
    
    circles = cv2.HoughCircles(image_gray, cv2.HOUGH_GRADIENT, 10, 150, minRadius=5, maxRadius=50)
    ret = circles[0, :]
    ret = np.column_stack((ret[0:,:2],ret[0:,3:]))
    return ret

In [ ]:
#Not working, could not find right approach
def screw_holes_termo(image, lower_bound, upper_bound):
    gray = cv2.cvtColor(to_3_channels(image_termo_norm), cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray,(5,5),0)
    edges = cv2.Canny(blurred, 6, 6)
    kernel = np.ones((2,2), np.uint8)

    #edges_e = cv2.erode(edges, kernel, iterations = 1)
    edges_d = cv2.dilate(edges, kernel, iterations = 1)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    edges_d_d = cv2.dilate(edges, kernel, iterations = 2)
    
    circles = cv2.HoughCircles(edges_d, cv2.HOUGH_GRADIENT, 5, 100, minRadius = 3, maxRadius = 15)
    img_rgb_cp = image_rgb.copy()
    
    #From stack overflow-----------
    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
 
        for (x, y, r) in circles:
            cv2.circle(img_rgb_cp, (x, y), r, (0, 255, 0), 4)
            cv2.rectangle(img_rgb_cp, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    #--------------------------------
    
    if DEBUG:
        plot_images(edges_d, img_rgb_cp)
       
    #return circles

In [ ]:
points_rgb = np.array(screw_holes_rgb(image_rgb, (70, 70, 130), (125, 135, 225)))
#order_points, BL is first and then counter-clockwise
points_rgb = order_points(points_rgb)    

#How I tried to resolve the holes for screws in termo image
if DEBUG:
    screw_holes_termo(image_termo, 1,1)

#points_termo = np.array(select_image_points(image_termo_norm))
points_termo = np.array([[122, 424], [304, 424], [301, 272], [120, 272]])

H, _ = cv2.findHomography(points_termo, points_rgb)
image_termo_warped = cv2.warpPerspective(image_termo, H, (1920, 1200))

plot_images(image_termo_warped, image_rgb)

# 4) Segmentujte v termosnímku
Pomocí rozsahu **reálných teplot** (ne jasových hodnot) vytvořte binární masku, která bude obsahovat **celé Raspberry Pi** bez připojených USB rezistorů. Masku zobrazte.

<div style="color: blue; text-align: right">[ 1,5 bodu ]</div>

In [ ]:
termo_maks = cv2.inRange(image_termo_warped, 37, 46)
kernel = np.ones((11,11),np.uint8)
termo_maks_closed = cv2.morphologyEx(termo_maks,cv2.MORPH_CLOSE, kernel)
contour_basic, _  = cv2.findContours(termo_maks, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) ###
contour_biggest = max(contour_basic, key=cv2.contourArea)

mask_biggest = cv2.drawContours(np.zeros_like(termo_maks), [contour_biggest], -1, color=(255, 0, 0), thickness=cv2.FILLED)

if DEBUG:
    plot_images(termo_maks, termo_maks_closed)
    
plot_images(mask_biggest)

### 5) Aplikujte data teplot na RGB obraz
Pomocí výše vytvořené **binární masky** segmentujte **data teplot**. Ty následně zobrazte pomocí vybrané [JET colormapy](https://docs.opencv.org/3.2.0/d3/d50/group__imgproc__colormap.html). Jako dílčí výstup **vytvořte funkci** pro **min-max normalizaci** teplotních hodnot obrazu, která přímá na vstupu **min, max, newmin, newmax**. Bude se hodit při aplikaci colormapy. Dále se vám může hodit funkce `to_3_channels()` k snadnějším bitovým operacím. Její využití však není podmínkou, způsobů získání výsledku je více.

Snímek by měl vypadat následovně (nenechte se zmást zobrazením BGR $\leftrightarrow$ RGB při používání matplotlibu).

<img src="images/fuse.png" width="50%"/>

Klíčová slova:
- min-max normalizace
- aplikace colormapy
- segmentace pomocí masky
- vážené spojení dvou obrazů

<div style="color: blue; text-align: right">[ 1,5 bodu ]</div>

In [ ]:
# Min-max normalization
def min_max_norm(img, old_min, old_max, new_min, new_max):
    scaled = np.zeros_like(img)
    for y in range(len(img)):
        for x in range(len(img[y])):
            #Mapping from old min - old max to new min - new max, pixel after pixel, didn't know other ways
            scaled[y][x] = min(old_max - (max(img[y][x] - new_min, old_min) / (new_max - new_min)) * old_max, old_max)
    
    return scaled.astype('uint8')

In [ ]:
temps_segmented = image_termo_warped.copy()
temps_segmented_norm = min_max_norm(temps_segmented, 0, 255, 37, 47)

#Apply color map
temps_segmented_norm = cv2.applyColorMap(temps_segmented_norm, cv2.COLORMAP_JET)
#Get rid of the noise
temps_segmented_norm = cv2.bitwise_and(to_3_channels(mask_biggest), temps_segmented_norm)

alpha = 0.5
beta = (1.0 - alpha)
gamma = 40

dst = cv2.addWeighted(temps_segmented_norm, alpha, image_rgb, beta, gamma, 0)
if DEBUG:
    plot_images(image_rgb, temps_segmented_norm)
    
plot_images(dst)

### 6) Segmentujte součástky
Využijte teplotní data k segmentaci **pouze 2 zahřívajících se** součástek - mikročipů. Zajímá nás plocha, která má teplotu vyšší než **43 °C**.

Při jejich hledání **využijte** informaci, že hledané mikročipy mají přibližně tvar čtverce.

<div style="color: blue; text-align: right">[ 1,5 bodu ]</div>

In [ ]:
def is_square(rect, tollerance = 0.2):
    w = rect[2]
    h = rect[3]
    max_r = (1 + tollerance)
    return h <= w * max_r and w <= h * max_r

In [ ]:
img_hot = cv2.inRange(image_termo_warped, 43, 80)
img_hot = cv2.bitwise_and(img_hot, mask_biggest)

contour_basic, _  = cv2.findContours(img_hot, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) ###\
#Check if its near square
contours_filtered_termo = [c for c in contour_basic if is_square(cv2.boundingRect(c))]

img_rgb_cp = image_rgb.copy()
img_rgb_cp_dbg = None

if DEBUG:
    img_rgb_cp_dbg = image_rgb.copy()
    for c in contour_basic:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(img_rgb_cp_dbg,(x,y),(x+w,y+h),(0,255,0),3)

for c in contours_filtered_termo:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(img_rgb_cp,(x,y),(x+w,y+h),(0,255,0),3)

if DEBUG:   
    plot_images(img_rgb_cp_dbg, img_rgb_cp)
else:    
    plot_images(img_rgb_cp)

### 7) Zjistěte velikost mikročipů v reálných jednotkách
Zjistěte pro každý mikročip, jak velká je celková zahřívaná plocha v cm$^2$, mající vyšší teplotu než je pro nás kritických **43 °C**. Pro přechod z pixelů do reálných jednotek jsme důkladně cvičili 2 různé způsoby. V tomto případě, je ale jeden z nich správnější, než ten druhý. Vyberte, který chcete, o absolutní přesnost zde v tuhle chvíli nejde. Disktutujte však správnost vybraného postupu.

**HINT -** Perspektiva.

<div style="color: blue; text-align: right">[ 1 bod ]</div>

In [ ]:
def cnt_dist(p1, p2):
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)**(0.5)

def cnt_area(rect, ratio):
    return round(rect[1][1] * rect[1][0] * ratio**2, 2)

In [ ]:
#lepší bude postup bez perspektivy, jelikož chybí vertikální osa.
p1 = (565, 197)
p2 = (996, 199)
real_lenght = 5
dst = cnt_dist(p1, p2)
ratio = real_lenght / dst

img_rgb_cp_cp = image_rgb.copy()

for c in contours_filtered_termo:
    rect = cv2.minAreaRect(c)
    text = str(round(cv2.contourArea(c) * ratio**2, 2)) + ' cm2'
    font = cv2.FONT_HERSHEY_SIMPLEX 
    org = (round(rect[0][0] - rect[1][0] / 2), round(rect[0][1] - rect[1][1]))
    
    img_rgb_cp_cp = cv2.putText(img_rgb_cp_cp, text, org, font, 1.2,  
                 (255, 0, 122), 3) 
    
img_rgb_cp_cp = cv2.drawContours(img_rgb_cp_cp, contours_filtered_termo,-1, (0,0,255), 3)

contours_filtered_termo = sorted(contours_filtered_termo, key=lambda x: cv2.contourArea(x))

assert(len(contours_filtered_termo) == 2)

chip_bigger_area_termo = cv2.contourArea(contours_filtered_termo[1]) * ratio**2
chip_smaller_area_termo = cv2.contourArea(contours_filtered_termo[0]) * ratio**2

print(f'Bigger chip overheated area: {round(chip_bigger_area_termo, 2)} cm2')
print(f'Smaller chip overheated area: {round(chip_smaller_area_termo, 2)} cm2')

plot_images(img_rgb_cp_cp)

In [ ]:
#premium 2

gray = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2GRAY)
gray = cv2.bitwise_and(gray, mask_biggest)
#Apply blur
blurred = cv2.GaussianBlur(gray,(3,3),0)
edges = cv2.Canny(blurred, 50, 140)

kernel = np.ones((2,2), np.uint8)
edges_d = cv2.dilate(edges, kernel, iterations = 2)
edges_d_masked = cv2.bitwise_not(edges_d)
edges_d_masked2 = cv2.bitwise_and(edges_d_masked, mask_biggest)

contour_basic, _  = cv2.findContours(edges_d, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE) ###\

min_area = 2000
#Check if its near square
contours_filtered = [c for c in contour_basic if is_square(cv2.boundingRect(c), 0.05) and cv2.contourArea(c) > min_area]
print(f'Total contours: {len(contour_basic)} vs. filtered contours: {len(contours_filtered)}')
img_rgb_cp = image_rgb.copy()

for c in contours_filtered:
    x,y,w,h = cv2.boundingRect(c)
    cv2.rectangle(img_rgb_cp,(x,y),(x+w,y+h),(0,255,0),3)
    
contours_filtered = sorted(contours_filtered, key=lambda x: cv2.contourArea(x))

if DEBUG:
    plot_images(edges_d, edges_d_masked, edges_d_masked2, img_rgb_cp)
else:
    plot_images(img_rgb_cp)

assert(len(contours_filtered) == 2)

chip_bigger_area_rgb = cv2.contourArea(contours_filtered[1]) * ratio**2
chip_smaller_area_rgb = cv2.contourArea(contours_filtered[0]) * ratio**2

print(f'Bigger chip area: {round(chip_bigger_area_rgb, 2)} cm2')
print(f'Smaller chip area: {round(chip_smaller_area_rgb, 2)} cm2')

print(f'Bigger chip overheated area: { round(chip_bigger_area_termo*100 / chip_bigger_area_rgb, 1) }%')
print(f'Smaller chip overheated area: { round (chip_smaller_area_termo*100 / chip_smaller_area_rgb, 1) }%')

---

### 8) Prémium #1
Nabízím 1 prémiový bod za detekci středů kružnic v RGB i v termo automatizovaně pomocí algoritmů. Například pomocí Houghovy transformace nebo i třeba hranové detekce + tvarových charakteristik. Jde primárně o díry na šrouby, které lze využít k hledání korespondenčních bodů a následné aplikaci perspektivní transformace.

### 9) Prémium #2
Nabízím 1 prémiový bod za výpočet plochy v cm$^2$ obou mikročipů v RGB obraze. Následně je nutné výsledky dát do poměru s vypočtenou plochou z termo snímku, která se zahřívala nad kritických 43° C. Tím lze například porovnat, který z nich se více zahřívá.